In [1]:
API_ID = "38926700"
API_HASH = "d0caea6d68df95cc7916c89d4d029c51"
YOUR_PHONE = "+79952368780"
PHONE_NUMBERS_FILE_NAME = 'my_phone_numbers.txt'
CHUNK_SIZE = 500 # How many numbers to process in one go
WAIT_SECONDS = 5 # How long to wait between chunks

In [2]:
!pip install telegram-phone-number-checker

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.9/568.9 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 6.9 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=67a1a87fd4eca18c5e3f0a7d85c4f52a65b95b2e51901bd4761401f38dd5b4e1
  Stored in directory: /root/.cache/pip/wheels/d9/43/32/ec313dd557e88e419489b9df40c04dad7b99576bde162496f8
  Created wheel for telethon: filename=telethon-1.33.1-py3-none-any.whl size=640987 sha256=7ebae9dc9193ed3571448405134bd1e6f963235785de9ce8583e2b63b3385abd
  Stored in director

In [3]:
!telegram-phone-number-checker --api-id $API_ID --api-hash "{API_HASH}" --api-phone-number "{YOUR_PHONE}" --phone-numbers "{+79952368780}"

INFO:root:Logging in...
INFO:telethon.network.mtprotosender:Connecting to 149.154.167.51:443/TcpFull...
INFO:telethon.network.mtprotosender:Connection to 149.154.167.51:443/TcpFull complete!
Enter the code (sent on telegram): 86635
INFO:root:Done.
INFO:root:Checking: phone_number='79952368780' ...
INFO:root:Done.
INFO:root:{
    "79952368780": {
        "error": "No response, the phone number is not on Telegram or has blocked contact adding."
    }
}
INFO:root:Results saved to results.json
INFO:telethon.network.mtprotosender:Disconnecting from 149.154.167.51:443/TcpFull...
INFO:telethon.network.mtprotosender:Disconnection from 149.154.167.51:443/TcpFull complete!


In [5]:
# Read phone numbers from the specified file
try:
    # PHONE_NUMBERS_FILE_NAME is expected to be 'my_phone_numbers.txt' from kernel state
    with open(PHONE_NUMBERS_FILE_NAME, 'r') as f:
        # Read all lines, strip whitespace, and filter out empty lines
        numbers = [line.strip() for line in f if line.strip()]
        # Ensure each number starts with a '+'
        numbers = [num if num.startswith('+') else '+' + num for num in numbers]
        # Join them with a comma for the command line argument
        PHONE_NUMBERS_TO_CHECK = ",".join(numbers)
    print(f"Loaded {len(numbers)} phone numbers from {PHONE_NUMBERS_FILE_NAME}.")
except FileNotFoundError:
    print(f"Error: The file '{PHONE_NUMBERS_FILE_NAME}' was not found. Please upload it to your Colab environment.")
    PHONE_NUMBERS_TO_CHECK = "" # Set to empty string to prevent errors in subsequent calls
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    PHONE_NUMBERS_TO_CHECK = ""

Loaded 2 phone numbers from my_phone_numbers.txt.


In [11]:
import subprocess
import json
import time
import os

output_file_name = 'results.json'
all_results = {}

# Check if the output file already exists and load previous results
try:
    with open(output_file_name, 'r') as f:
        all_results = json.load(f)
    print(f"Loaded existing results from {output_file_name}.")
except (FileNotFoundError, json.JSONDecodeError):
    print(f"No existing results found in {output_file_name}. Starting fresh.")


# Ensure numbers is a list, not a comma-separated string
if isinstance(PHONE_NUMBERS_TO_CHECK, str):
    numbers_list = [num.strip() for num in PHONE_NUMBERS_TO_CHECK.split(',') if num.strip()]
else:
    numbers_list = numbers # Assuming 'numbers' is already a list from a previous cell


# Process phone numbers in chunks
for i in range(0, len(numbers_list), CHUNK_SIZE):
    chunk = numbers_list[i:i + CHUNK_SIZE]
    print(f"\nProcessing chunk {i // CHUNK_SIZE + 1}/{(len(numbers_list) - 1) // CHUNK_SIZE + 1} ({len(chunk)} numbers)...")

    # Convert the chunk list back to a comma-separated string for the command line argument
    chunk_str = ",".join(chunk)

    try:
        print(f"Attempting to process chunk with numbers: {chunk_str[:100]}...") # Truncate for display

        # Construct the command for the external tool
        command_args = [
            "telegram-phone-number-checker",
            "--api-id", str(API_ID),
            "--api-hash", API_HASH,
            "--api-phone-number", YOUR_PHONE,
            "--phone-numbers", chunk_str
        ]

        # Execute the command. The tool saves results to 'results.json'
        # We assume the session file exists from previous interactive login and no further prompts are needed.
        process = subprocess.run(command_args, capture_output=True, text=True, check=False) # check=False to handle potential non-zero exit codes gracefully

        if process.returncode != 0:
            print(f"Command failed with exit code {process.returncode} for chunk {i // CHUNK_SIZE + 1}.")
            print(f"Stderr: {process.stderr}")
            print(f"Stdout: {process.stdout}")
            # If the command failed, we might not have updated results.json for this chunk.
            # We will proceed without updating all_results for this failed chunk.
        else:
            # The tool successfully ran and *should* have updated results.json
            # Load the results from the file, which the CLI tool overwrites.
            if os.path.exists(output_file_name):
                with open(output_file_name, 'r') as f:
                    chunk_specific_results = json.load(f)
                all_results.update(chunk_specific_results)
                print(f"Aggregated results for {len(chunk_specific_results)} numbers in this chunk.")
            else:
                print(f"Warning: '{output_file_name}' not found after command execution for chunk {i // CHUNK_SIZE + 1}.")


        # Save the cumulative all_results after each chunk processed
        with open(output_file_name, 'w') as f:
            json.dump(all_results, f, indent=4)
        print(f"Cumulative results saved to {output_file_name} after chunk {i // CHUNK_SIZE + 1}.")

    except Exception as e:
        print(f"An unexpected error occurred while processing chunk {i // CHUNK_SIZE + 1}: {e}")
        # Save current progress even if an unexpected error occurs
        with open(output_file_name, 'w') as f:
            json.dump(all_results, f, indent=4)
        print(f"Partial results saved to {output_file_name} before continuing.")

    # Wait for the specified time before processing the next chunk
    if i + CHUNK_SIZE < len(numbers_list): # Don't wait after the last chunk
        print(f"Waiting for {WAIT_SECONDS} seconds before next chunk...")
        time.sleep(WAIT_SECONDS)

print(f"\nFinished processing all phone numbers. Final results saved to {output_file_name}.")

Loaded existing results from results.json.

Processing chunk 1/1 (2 numbers)...
Attempting to process chunk with numbers: +79952368780,+79939681829...
Aggregated results for 2 numbers in this chunk.
Cumulative results saved to results.json after chunk 1.

Finished processing all phone numbers. Final results saved to results.json.
